In [20]:
!pip install langchain
!pip install langchain-community

In [21]:
!pip install pypdf

In [22]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader
import os
docs = []
my_dir = '/content/books'
for f in os.listdir(my_dir):
  if not f.endswith(".pdf"):
    continue
  print(f)
  loader = PyPDFLoader(os.path.join(my_dir,f))
  doc = loader.load()
  print(type(doc),len(doc))
  print(type(doc[0]))
  print(doc[0].metadata)
  print(len(doc[0].page_content))
  docs.extend(doc)
print(len(docs))

keph104.pdf
<class 'list'> 22
<class 'langchain_core.documents.base.Document'>
{'source': '/content/books/keph104.pdf', 'page': 0}
2204
22


In [23]:
!pip install -qU langchain-text-splitters

In [24]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=100)
chunks = splitter.split_documents(docs)
print(f"{len(docs)} documents have been split into {len(chunks)} chunks")
print(f"Each chunk is of type {type(chunks[0])}")
print(f"\nSample chunk page content:\n{chunks[0].page_content}")

22 documents have been split into 90 chunks
Each chunk is of type <class 'langchain_core.documents.base.Document'>

Sample chunk page content:
CHAPTER  FOUR
LAWS OF MOTION
4.1  INTRODUCTION
In the preceding Chapter , our concer n was to describe the
motion of a particle in space quantitatively. W e saw that
uniform motion needs the concept of velocity alone whereas
non-uniform motion requires the concept of acceleration in
addition.  So far , we have not asked the question as to what
gover ns the motion of bodies. In this chapter , we tur n to this
basic question.
Let us first guess the answer based on our common
experience. To move a football at rest, someone must kick it.
To throw a stone upwards, one has to  give it an upward
push.  A breeze causes the branches of a tree to swing; a
strong wind can even move heavy objects. A boat moves in a
flowing river without anyone rowing it. Clearly, some external
agency is needed to provide force to move a body from rest.
Likewise, an external

In [25]:
!pip install -qU sentence_transformers

In [26]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name = "all-MiniLM-L6-v2")

In [27]:
!pip install -qU langchain-chroma

In [28]:
from langchain_chroma import Chroma

persist_directory = os.path.join(my_dir, "chroma_db")

vectordb = Chroma.from_documents(
    documents=chunks,
    persist_directory=persist_directory,
    embedding=embeddings
)

In [29]:
query = "What is laws of motion "
relevant_docs = vectordb.max_marginal_relevance_search(query)
print(type(relevant_docs),len(relevant_docs))
print(type(relevant_docs[0]))


<class 'list'> 4
<class 'langchain_core.documents.base.Document'>


In [30]:
for d in relevant_docs:
  print(f"------------{len(d.page_content)}------- \n{d.page_content}")

------------961------- 
CHAPTER  FOUR
LAWS OF MOTION
4.1  INTRODUCTION
In the preceding Chapter , our concer n was to describe the
motion of a particle in space quantitatively. W e saw that
uniform motion needs the concept of velocity alone whereas
non-uniform motion requires the concept of acceleration in
addition.  So far , we have not asked the question as to what
gover ns the motion of bodies. In this chapter , we tur n to this
basic question.
Let us first guess the answer based on our common
experience. To move a football at rest, someone must kick it.
To throw a stone upwards, one has to  give it an upward
push.  A breeze causes the branches of a tree to swing; a
strong wind can even move heavy objects. A boat moves in a
flowing river without anyone rowing it. Clearly, some external
agency is needed to provide force to move a body from rest.
Likewise, an external force is needed also to retard or stop
motion.  Y ou can stop a ball r olling down an inclined plane by
------------36

In [31]:
!pip install -qU langchain-groq

In [32]:
GROQ_API_KEY = "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"

In [33]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

# GROQ_API_KEY = <your_api_key>

llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0,
    api_key=GROQ_API_KEY
)

system_message = "You are a helpful assistant. Answer the user's query based on the given context. If the query cannot be answered from the given context, simply say so. DO NOT make up answers."

human_message = """Answer the following question based on the given context:
{question}

Context: {context}"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("human", human_message)
])

chain = prompt | llm

In [34]:
# relevant_docs is a list of Document objects
context = "\n".join([d.page_content for d in relevant_docs])

response = chain.invoke({"question": query, "context": context})
print(response)

content="Based on the given context, the laws of motion are not explicitly defined. However, the chapter appears to be introducing the concept of laws of motion and will likely discuss Newton's laws of motion, which include the law of inertia, Newton's first law of motion, Newton's second law of motion, and Newton's third law of motion. Additionally, the chapter may also cover other related concepts such as conservation of momentum, equilibrium of a particle, and common forces in mechanics." response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 873, 'total_tokens': 969, 'completion_time': 0.295489572, 'prompt_time': 0.094752215, 'queue_time': None, 'total_time': 0.390241787}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_87cbfbbc4d', 'finish_reason': 'stop', 'logprobs': None} id='run-6ee74c05-851c-4948-8a2e-653d3b026713-0' usage_metadata={'input_tokens': 873, 'output_tokens': 96, 'total_tokens': 969}
